In [1]:
#!conda install transformers

import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
#import tensorflow_hub as hub
from transformers import BertTokenizer
#!pip install tensorflow_text
#import tensorflow_text as text
import numpy as np
df = pd.read_csv('preprocessed_lyrics.csv')
SIZE = 1000
df = df.groupby('Genre', group_keys=False).apply(lambda s: s.sample(SIZE, random_state=42)) #


In [2]:
df

,SName,Lyric,Genre,Artist,lyric_length_processed
2157,Number One,uh uh uh I got to bring dirty thas you well wa...,Hip Hop,Nelly,299
3187,Put That Thang On U (ft. Snoop Dogg),Fala lady like lady Ne Yo sing to real quick I...,Hip Hop,Ne-yo,233
9164,The Resurrection of Scott Mescudi,Instrumental once realize free you fly,Hip Hop,Kid Cudi,6
2145,Feel Like Home,these day dark night cold People act like lose...,Hip Hop,Fort Minor,239
6442,Public Service Announcement 2000,this another public bring in by Slim Shady Emi...,Hip Hop,Eminem,51
...,...,...,...,...,...
36875,The Night Time Is The Right Time,you know oh right to love I say ooh right to l...,Rock,Creedence Clearwater Revival,59
38526,"Sphinx, Pt. I",wander travel aimlessly blister lose mirage re...,Rock,Lucas Ray Exp,293
32623,Vinheta Final Por Mauro E Quitéria,instrumental Warner Chappell direto,Rock,Titãs,4
36551,"Me You, We Two",I we try begin I we try begin and blow but wil...,Rock,Blur,18


In [ ]:
# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
assert 'GPU' in str(device_lib.list_local_devices())

# confirm Keras sees the GPU (for TensorFlow 1.X + Keras)
from keras import backend
assert len(backend.tensorflow_backend._get_available_gpus()) > 0



In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) # list of DeviceAttributes



### Initializing empty arrays for storing tokenized text

In [3]:

possible_labels = df.Genre.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

df['label'] = df.Genre.replace(label_dict)

In [4]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(df.Lyric.values, 
                                                df.label.values, 
                                                random_state=42, 
                                                test_size=0.1, 
                                                stratify=df.label.values)

In [5]:
# set array dimensions
seq_len = 512 # Our input sequences can't be too long so we set it to max 512 tokens
num_samples = len(xtrain) 

# initialize empty zero arrays
Xids = np.zeros((num_samples, seq_len))
Xmask = np.zeros((num_samples, seq_len))

# check shape
print(Xids.shape)

# initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

for i, lyric in enumerate(xtrain):
    tokens = tokenizer.encode_plus(lyric, max_length=seq_len, truncation=True, #truncation_True -> text longer than 512 tokens get compressed
                                   padding='max_length', add_special_tokens=True, #for text shorter than 512, pad to length 512. Special tokens adds [CLS],[SEP],[PAD]
                                   return_tensors='tf')
    # assign tokenized outputs to respective rows in numpy arrays
    Xids[i, :] = tokens['input_ids']
    Xmask[i, :] = tokens['attention_mask']

(2700, 512)


101 is for [CLS] "start of seq", 0 is for [PAD] , 

In [6]:
Xids

array([[  101., 18334., 18334., ...,     0.,     0.,     0.],
       [  101.,  1184.,  1165., ...,     0.,     0.,     0.],
       [  101.,  1165.,   157., ...,     0.,     0.,     0.],
       ...,
       [  101.,  4804.,  2671., ...,     0.,     0.,     0.],
       [  101.,  2100.,  1435., ...,     0.,     0.,     0.],
       [  101.,  9294.,  1187., ...,     0.,     0.,     0.]])

In [7]:
Xmask

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

Xmask is a control for the attention layer in bert. whereever xmask is 1 Bert will calculate the attention for that token. To avoid for Bert making connections with padding tokens



### one-hot encoding the Genres

In [8]:

# first extract sentiment column
arr = ytrain
# we then initialize the zero array
labels = np.zeros((num_samples, arr.max()+1))

# set relevant index for each row to 1 (one-hot encode)
labels[np.arange(num_samples), arr] = 1
print(labels)

[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


### Building a tf.data.Dataset object using the input and label tensors. Then transforming them into the correct format for the model.

In [9]:
# create the dataset object
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

def map_func(input_ids, masks, labels):
    # we convert our three-item tuple into a two-item tuple where the input item is a dictionary
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

# then we use the dataset map method to apply this transformation
dataset = dataset.map(map_func)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [10]:
# we will split into batches of 16
batch_size = 16

# shuffle and batch - dropping any remaining samples that don't cleanly
# fit into a batch of 16
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)
dataset.take(1) # We now have 16 samples per batch

<TakeDataset shapes: ({input_ids: (16, 512), attention_mask: (16, 512)}, (16, 3)), types: ({input_ids: tf.float64, attention_mask: tf.float64}, tf.float64)>

In [11]:
# set split size (90% training data) and calculate training set size
split = 1
size = int((Xids.shape[0]/batch_size)*split)
print(size)
print('number of samples in training ' + str(56*16))
# get training and validation sets
train_ds = dataset.take(size)

168
number of samples in training 896


In [12]:
# AutoModel for PyTorch, TFAutoModel for TensorFlow
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [13]:
import tensorflow as tf

# two input layers, we ensure layer name variables match to dictionary keys in TF dataset
input_ids = tf.keras.layers.Input(shape=(512,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(512,), name='attention_mask', dtype='int32')

# we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)
embeddings = bert.bert(input_ids, attention_mask=mask)[1]  # access final activations with [0]

# convert bert embeddings into 5 output classes
x = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
y = tf.keras.layers.Dense(3, activation='softmax', name='outputs')(x)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [14]:
# initialize model
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# (optional) freeze bert layer
model.layers[2].trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [15]:
optimizer = tf.keras.optimizers.Adam(lr=1e-5, decay=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

/home/ubuntu/anaconda3/envs/tensorflow_p37/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [16]:
history = model.fit(
    train_ds,
    epochs=2
)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
168/168 [==============================] - 113s 642ms/step - loss: 1.1013 - accuracy: 0.3661
Epoch 2/2
168/168 [==============================] - 108s 644ms/step - loss: 1.0585 - accuracy: 0.4468


In [17]:
def prep_data(text):
    tokens = tokenizer.encode_plus(text, max_length=512,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_token_type_ids=False,
                                   return_tensors='tf')
    # tokenizer returns int32 tensors, we need to return float64, so we use tf.cast
    return {'input_ids': tf.cast(tokens['input_ids'], tf.float64),
            'attention_mask': tf.cast(tokens['attention_mask'], tf.float64)}

is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.


In [19]:
pred = [model.predict(prep_data(lyric) for lyric in xtest)]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [20]:
y = ytest


In [21]:
pred = np.array(pred)
pred = [item for sublist in pred for item in sublist]
y_pred_bool = np.argmax(pred, axis=1)
y_pred_bool

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1,
       1, 0, 0, 0, 1, 2, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 1, 1, 0,
       0, 0, 0, 2, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 0, 2, 0, 0, 1, 1, 1, 0, 0, 0, 2, 1, 0, 1, 2,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 2, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 2, 0, 0, 2, 1, 0, 1, 0, 0])

In [22]:
from sklearn.metrics import classification_report

print(classification_report(y, y_pred_bool, target_names=label_dict))

              precision    recall  f1-score   support

     Hip Hop       0.41      0.89      0.56       100
         Pop       0.41      0.28      0.33       100
        Rock       0.47      0.07      0.12       100

    accuracy                           0.41       300
   macro avg       0.43      0.41      0.34       300
weighted avg       0.43      0.41      0.34       300

